# PiP & Import

In [ ]:
!pip install langchain
!pip install chromadb
!pip install sentence-transformers

In [50]:
!pip install faiss-gpu # For CUDA 7.5+ Supported GPU's.
# OR
!pip install faiss-cpu # For CPU Installation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 86.4 MB/s eta 0:00:00


In [51]:
import os
import sys
import langchain
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

#Embeddings


Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/dataset/dataset_v3.csv")

In [ ]:
# df = df['cases_text']

In [ ]:
#df.to_csv('/content/drive/MyDrive/dataset/cases_text.csv', index=False)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/dataset/cases_text.csv')

In [5]:
df

,cases_text
0,على رسول ﷲ أما بعد:فلدى الدائرة التجارية الثام...
1,على رسول ﷲ أما بعد:فلدى الدائرة التجارية الثان...
2,على رسول ﷲ أما بعد: فلدى الدائرة التجارية الأو...
3,على رسول ﷲ أما بعد:فلدى الدائرة التجارية الاول...
4,على رسول ﷲ أما بعد:فلدى الدائرة التجارية الاول...
...,...
2223,رسول ﷲ الدائرة التجارية الواحد والون وبناء الق...
2224,رسول ﷲ الدائرة التجارية الاولى وبناء القضية رق...
2225,رسول ﷲ الدائرة التجارية الخامسة وبناء القضية ر...
2226,رسول ﷲ الدائرة التجارية الثانية وبناء القضية ر...


In [11]:
#split data by loader
loader = CSVLoader(file_path="/content/drive/MyDrive/dataset/cases_text.csv",)

data = loader.load()

In [9]:
data

Output hidden; open in https://colab.research.google.com to view.

In [12]:
# create the Hugging Face embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [13]:
# load it into ChromaDb
db = Chroma.from_documents(data, embedding_function)

KeyboardInterrupt: ignored

## Vectorstores

In [6]:
from langchain.vectorstores import Chroma
#create directory -> DB embedding
persist_directory = '/content/drive/MyDrive/dataset/docs/chroma'

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [14]:
# create vectordb
vectordb = Chroma.from_documents(
    documents=data,
    embedding=embedding_function,
    persist_directory=persist_directory
)

KeyboardInterrupt: ignored

In [ ]:
#print chunk number
print(vectordb._collection.count())

2228


#vectordb

In [15]:
persist_directory = '/content/drive/MyDrive/dataset/docs/chroma'

In [16]:
vectordb = Chroma(
    embedding_function=embedding_function,
    persist_directory=persist_directory
)

In [17]:
print(vectordb._collection.count())

2228


In [18]:
vectordb.persist()

In [52]:
db = FAISS.from_documents(data, embedding_function)

In [65]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming your input vector is 'input_vector' and vectordb contains your embedded vectors
input_vector = embedding_QUE # Use your actual input data here
vectordb_vectors = vectordb.get_all_vectors()  # Assuming there's a method to get all vectors from vectordb

# Convert vectors to NumPy arrays for compatibility
input_vector = np.array(input_vector).reshape(1, -1)  # Reshape to ensure it's a 2D array
vectordb_vectors = np.array(vectordb_vectors)

# Calculate cosine similarity
similarity_scores = cosine_similarity(input_vector, vectordb_vectors)

# Print or use the similarity scores as needed
print(similarity_scores)


AttributeError: ignored

Similarity Search

In [39]:
question = " قضايا الايدي العاملة ؟ "

In [40]:
embedding_QUE = embedding_function.embed_query(question)

In [ ]:
embedding_QUE

In [58]:
docs = db.similarity_search_by_vector(embedding_QUE,k=3)

In [59]:
len(docs)

3

In [57]:
docs = db.similarity_search(question,k=5)

In [60]:
for doc in docs:
    print(doc.metadata)

{'source': '/content/drive/MyDrive/dataset/cases_text.csv', 'row': 1638}
{'source': '/content/drive/MyDrive/dataset/cases_text.csv', 'row': 1835}
{'source': '/content/drive/MyDrive/dataset/cases_text.csv', 'row': 1840}


In [61]:
docs[0]

Document(page_content='cases_text: على رسول ﷲ أما بعد: فلدى الدائرة التجارية الثامنة وبناء على القضية رقم ١٤٤١ لعام ١٤٣٩ ه المقامة من/(...) هوية وطنية(...) المقامة من/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) ضد/(...) هوية وطنية(...) القاضي نايف العنزي القاضي بن بن سليمان الحيلان القاضي بن بن صالح الغامدي (الوقائع) تتلخص وقائع هذه الدعوى بالقدر اللازم للفصل فيها أنه قد وردت إلى المحكمة التجارية بجدة صحيفة دعوى مقدمة من المدعية ونصها (حيث أن موكلي (المدعين) من ضمن ورثة المتوفي /(...) (رحمه ﷲ) (...) الجنسية بموجب السجل المدني رقم (...) والمتوفي في تاريخ ٣ / ٦ / ورثته في أطراف هذه الدعوى دون غيرهم وذلك بموجب صك حصر الورثة رقم (...) بتاريخ ١٨ / ٧ / سبق وأن تقدمنا بدعوى قضائية لدى محكمة الأحوال الشخصية بمحافظة جده لحصر وتثمين التركة ومن ثم تقسيمها على جميع الورثة حسب الأنصبة الشرعية وكانت 